# Working with Tropospheric Emissions: Monitoring of Pollution (TEMPO)'s Image Service in ipyleaflet

## Export Image

In [5]:
import requests
from ipyleaflet import Map, ImageOverlay, WidgetControl, projections, Rectangle
from ipywidgets import Dropdown, VBox
from datetime import datetime
import time
import json
import urllib.parse

def convert_to_milliseconds(date_time_str):
    """Converts a date-time string in 'YYYY-MM-DD HH:MM:SS' format to milliseconds since epoch."""
    dt = datetime.strptime(date_time_str, '%Y-%m-%d %H:%M:%S')
    milliseconds_since_epoch = int(dt.timestamp() * 1000)
    print(milliseconds_since_epoch)
    return milliseconds_since_epoch


# Function to get the JSON request of the image using the image export service
def response_export_image(image_service_url, bbox, date_time):
    
    """For more info on how to adjuste parameters, visit https://developers.arcgis.com/rest/services-reference/enterprise/export-image/"""

    # Provided rendering rule for the color ramp
    rendering_rule = {
        "rasterFunctionArguments": {
            "ColorrampName": "Temperature",
            "Raster": {
                "rasterFunctionArguments": {
                    "StretchType": 5,
                    "Statistics": [[0, 30000000000000000, 910863682171422.1, 9474291611234248]],
                    "DRA": False,
                    "UseGamma": False,
                    "Gamma": [1],
                    "ComputeGamma": True,
                    "Min": 0,
                    "Max": 255
                },
                "rasterFunction": "Stretch",
                "outputPixelType": "U64",
                "variableName": "Raster"
            }
        },
        "rasterFunction": "Colormap",
        "variableName": "Raster"
    }

    # Convert rendering rule to JSON string
    rendering_rule_json = json.dumps(rendering_rule)
  
    print(rendering_rule)

    params = {
        'bbox': ','.join(map(str, bbox)),
        'bboxSR': '4326', #4326
        'imageSR': '4326',#3857
        'size': '2000,1000',
        'time': '',
        'format': 'jpgpng',
        'pixelType': 'U64',
        'noData': '',
        'noDataInterpretation': 'esriNoDataMatchAny',
        'interpolation': 'RSP_BilinearInterpolation',
        'compression': '',
        'compressionQuality': '',
        'bandIds': '',
        'sliceId': '1000',
        'mosaicRule': '',
        'renderingRule': rendering_rule_json,
        'adjustAspectRatio': 'true',
        'validateExtent': 'false',
        'lercVersion': '',
        'compressionTolerance': '',
        'f': 'json',
        'variableName': 'T2M'
    }

    response = requests.get(image_service_url, params=params)
    response.raise_for_status()
    print(response.json())
    return response.json()


image_service_url = "https://gis.earthdata.nasa.gov/image/rest/services/C2930763263-LARC_CLOUD/TEMPO_NO2_L3_V03_HOURLY_TROPOSPHERIC_VERTICAL_COLUMN_BETA/ImageServer/exportImage"

# User inputs defaults
date_time_str = "2024-07-10 9:16:57" #EST time #TODO
bbox = [-125.0, 22, -66, 51]

# Convert time to milliseconds
date_time = convert_to_milliseconds(date_time_str) # equals 1720617417000 in ms since epoch

# retrieves images given user input
response = response_export_image(image_service_url, bbox, date_time)

print(response)
print(bbox)
xmin , ymin, xmax, ymax = response['extent']['xmin'] ,  response['extent']['ymin'] ,  response['extent']['xmax'] ,  response['extent']['ymax']
# xmin , ymin, xmax, ymax = bbox

center_lat = (ymin + ymax) / 2
center_lon = (xmin + xmax) / 2

print('CENTROS', center_lat , center_lon)
print (xmin , ymin, xmax, ymax)

# Initialize the map
m = Map(center=(center_lat, center_lon), zoom=4, chr=projections.EPSG4326 )

# Add new image layer
bounds = [[ymin, xmin], [ymax, xmax]]
print(bounds)
image_overlay = ImageOverlay(url=response['href'], bounds=bounds, opacity=0.5)

# rectangle = Rectangle(bounds=(bounds), 
#                       color="blue", weight=2)
# m.add_layer(rectangle)
m.add_layer(image_overlay)
print(bbox)


m



1720617417000
{'rasterFunctionArguments': {'ColorrampName': 'Temperature', 'Raster': {'rasterFunctionArguments': {'StretchType': 5, 'Statistics': [[0, 30000000000000000, 910863682171422.1, 9474291611234248]], 'DRA': False, 'UseGamma': False, 'Gamma': [1], 'ComputeGamma': True, 'Min': 0, 'Max': 255}, 'rasterFunction': 'Stretch', 'outputPixelType': 'U64', 'variableName': 'Raster'}}, 'rasterFunction': 'Colormap', 'variableName': 'Raster'}
{'href': 'https://gis.earthdata.nasa.gov/image/rest/directories/arcgisoutput/C2930763263-LARC_CLOUD/TEMPO_NO2_L3_V03_HOURLY_TROPOSPHERIC_VERTICAL_COLUMN_BETA_ImageServer/_ags_ab1cc476_a785_499f_8614_91adfe204ac9.png', 'width': 2000, 'height': 1000, 'extent': {'xmin': -125, 'ymin': 21.75, 'xmax': -66, 'ymax': 51.25, 'spatialReference': {'wkid': 4326, 'latestWkid': 4326}}, 'scale': 0}
{'href': 'https://gis.earthdata.nasa.gov/image/rest/directories/arcgisoutput/C2930763263-LARC_CLOUD/TEMPO_NO2_L3_V03_HOURLY_TROPOSPHERIC_VERTICAL_COLUMN_BETA_ImageServer/_ags

Map(center=[36.5, -95.5], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_ou…

In [6]:















# # Connect the image update to the time dropdown change
# time_dropdown.observe(update_image, 'value')

# # Update the image when the map is moved or zoomed
# m.observe(update_image, 'zoom')
# m.observe(update_image, 'center')

# # Add the widgets to the map
# widget_control = WidgetControl(widget=VBox([time_dropdown]), position='topright')
# m.add_control(widget_control)

# # Update the image initially
# update_image()

# # Display the map
# m


#  {
#   "sliceId": 1000,
#   "multidimensionalDefinition": [{
#    "variableName": "NO2 Troposphere",
#    "dimensionName": "StdTime",
#    "values": [1720617417000]
#   }]
#  },


# https://gis.earthdata.nasa.gov/image/rest/services/C2930763263-LARC_CLOUD/
# TEMPO_NO2_L3_V03_HOURLY_TROPOSPHERIC_VERTICAL_COLUMN_BETA/ImageServer/exportImage
# ?bbox=-168.00000549350375,14.00000081336853,-13.00001324350373,72.99999786336853